In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras

In [2]:
simi = tf.saved_model.load('saved_model_pre50sk0')

In [6]:
outputs = simi(inputs)

NameError: name 'inputs' is not defined

In [2]:
outputs = keras.applications.ResNet50(weights=None,include_top=False, input_shape=(330,330,3))

In [4]:
outputs.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 330, 330, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 336, 336, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 165, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 165, 165, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
outputs.input_3

In [ ]:
#https://stackoverflow.com/questions/59705507/how-to-access-and-visualize-the-weights-in-a-pre-trained-tensorflow-2-model/59707063#59707063

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.resnet import resnet50
from torchsummary import summary


class Model(nn.Module):
    def __init__(self, feature_dim=128):
        super(Model, self).__init__()

        self.f = []
        for name, module in resnet50().named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if not isinstance(module, nn.Linear) and not isinstance(module, nn.MaxPool2d):
                self.f.append(module)
        # encoder
        self.f = nn.Sequential(*self.f)
        # projection head
        #self.g = nn.Sequential(nn.Linear(2048, 512, bias=False), nn.BatchNorm1d(512),
        #                       nn.ReLU(inplace=True), nn.Linear(512, feature_dim, bias=True))

    def forward(self, x):
        x = self.f(x)
        feature = torch.flatten(x, start_dim=1)
        #out = self.g(feature)
        return F.normalize(feature, dim=-1)

In [3]:
model = Model(128)

In [9]:
summary(model, (3, 330, 330))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 2048, 1, 1]          --
|    └─Conv2d: 2-1                       [-1, 64, 330, 330]        1,728
|    └─BatchNorm2d: 2-2                  [-1, 64, 330, 330]        128
|    └─ReLU: 2-3                         [-1, 64, 330, 330]        --
|    └─Sequential: 2-4                   [-1, 256, 330, 330]       --
|    |    └─Bottleneck: 3-1              [-1, 256, 330, 330]       75,008
|    |    └─Bottleneck: 3-2              [-1, 256, 330, 330]       70,400
|    |    └─Bottleneck: 3-3              [-1, 256, 330, 330]       70,400
|    └─Sequential: 2-5                   [-1, 512, 165, 165]       --
|    |    └─Bottleneck: 3-4              [-1, 512, 165, 165]       379,392
|    |    └─Bottleneck: 3-5              [-1, 512, 165, 165]       280,064
|    |    └─Bottleneck: 3-6              [-1, 512, 165, 165]       280,064
|    |    └─Bottleneck: 3-7              [-1, 512, 165

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 2048, 1, 1]          --
|    └─Conv2d: 2-1                       [-1, 64, 330, 330]        1,728
|    └─BatchNorm2d: 2-2                  [-1, 64, 330, 330]        128
|    └─ReLU: 2-3                         [-1, 64, 330, 330]        --
|    └─Sequential: 2-4                   [-1, 256, 330, 330]       --
|    |    └─Bottleneck: 3-1              [-1, 256, 330, 330]       75,008
|    |    └─Bottleneck: 3-2              [-1, 256, 330, 330]       70,400
|    |    └─Bottleneck: 3-3              [-1, 256, 330, 330]       70,400
|    └─Sequential: 2-5                   [-1, 512, 165, 165]       --
|    |    └─Bottleneck: 3-4              [-1, 512, 165, 165]       379,392
|    |    └─Bottleneck: 3-5              [-1, 512, 165, 165]       280,064
|    |    └─Bottleneck: 3-6              [-1, 512, 165, 165]       280,064
|    |    └─Bottleneck: 3-7              [-1, 512, 165

In [1]:
from torchvision.models.resnet import resnet50
for name, module in resnet50().named_children():
    print(module)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace=True)
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, t

In [ ]:
summary(resnet50, (3, 330, 330))